# 🔄 Sorting Algorithms Master Guide

**Phase 3 (Interview Preparation) - Algorithms Module 1**

**Prerequisites**: Basic programming knowledge

Master fundamental sorting algorithms with deep implementation and analysis!

---

## 📊 Sorting Algorithm Complexity Summary

| Algorithm | Best | Average | Worst | Space | Stable |
|-----------|------|---------|-------|-------|--------|
| **Bubble Sort** | O(n²) | O(n²) | O(n²) | O(1) | Yes |
| **Selection Sort** | O(n²) | O(n²) | O(n²) | O(1) | No |
| **Insertion Sort** | O(n) | O(n²) | O(n²) | O(1) | Yes |
| **Merge Sort** | O(n log n) | O(n log n) | O(n log n) | O(n) | Yes |
| **Quick Sort** | O(n log n) | O(n log n) | O(n²) | O(log n) | No |
| **Heap Sort** | O(n log n) | O(n log n) | O(n log n) | O(1) | No |
| **Counting Sort** | O(n+k) | O(n+k) | O(n+k) | O(k) | Yes |
| **Radix Sort** | O(n*d) | O(n*d) | O(n*d) | O(n+radix) | Depends |

In [ ]:
// Utility functions for all sorting implementations
object SortUtils {
  def swap(arr: Array[Int], i: Int, j: Int): Unit = {
    val temp = arr(i)
    arr(i) = arr(j)
    arr(j) = temp
  }
  
  def printArray(arr: Array[Int], prefix: String = "Array"): Unit = {
    println(s"$prefix: [${arr.mkString(", ")}]")
  }
  
  def isSorted(arr: Array[Int]): Boolean = {
    arr.zip(arr.tail).forall { case (a, b) => a <= b }
  }
  
  // Generate test arrays
  def randomArray(size: Int, max: Int = 100): Array[Int] = {
    Array.fill(size)(scala.util.Random.nextInt(max) + 1)
  }
  
  val testArrays = Map(
    "small" -> Array(64, 34, 25, 12, 22, 11, 90),
    "empty" -> Array[Int](),
    "single" -> Array(42),
    "sorted" -> Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
    "reverse" -> Array(10, 9, 8, 7, 6, 5, 4, 3, 2, 1),
    "duplicates" -> Array(3, 1, 4, 1, 5, 9, 2, 6, 5, 3)
  )
}

// Import utilities
import SortUtils._

## 🫧 Bubble Sort (O(n²) - Stable)

Repeatedly swaps adjacent elements if they are in wrong order.

In [ ]:
// Bubble Sort: Optimized version with early stopping
def bubbleSort(arr: Array[Int]): Array[Int] = {
  if (arr.length <= 1) return arr.clone()
  
  val result = arr.clone()
  var swapped = true
  var end = arr.length - 1
  
  while (swapped && end > 0) {
    swapped = false
    
    for (i <- 0 until end) {
      if (result(i) > result(i + 1)) {
        swap(result, i, i + 1)
        swapped = true
      }
    }
    
    end -= 1  // Optimization: reduce range each pass
  }
  
  result
}

// Bubble Sort: Counting swaps version
def bubbleSortWithCount(arr: Array[Int]): (Array[Int], Int) = {
  val result = arr.clone()
  var swapCount = 0
  
  for (i <- 0 until result.length - 1) {
    for (j <- 0 until result.length - 1 - i) {
      if (result(j) > result(j + 1)) {
        swap(result, j, j + 1)
        swapCount += 1
      }
    }
  }
  
  (result, swapCount)
}

// Test Bubble Sort
println("=== Bubble Sort Tests ===")
val testArray = testArrays("small").clone()
printArray(testArray, "Original")

val sortedBubble = bubbleSort(testArray)
printArray(sortedBubble, "Bubble Sorted")
println(s"Is sorted: ${isSorted(sortedBubble)}")

val (sortedWithCount, swaps) = bubbleSortWithCount(testArrays("small"))
println(s"Total swaps: $swaps")

// Test various cases
testArrays.foreach { case (name, arr) =>
  val sorted = bubbleSort(arr.clone())
  println(s"$name array: sorted=${isSorted(sorted)}, length=${arr.length}")
}
println()

## 🎯 Quick Sort (O(n log n) avg, O(n²) worst - NOT stable)

Divide and conquer algorithm - most practical general-purpose sort.

In [ ]:
// Quick Sort with different pivot strategies
object QuickSort {
  
  // Lomuto partition scheme (fast, but not stable)
  def partitionLomuto(arr: Array[Int], low: Int, high: Int, pivotIndex: Int): Int = {
    val pivot = arr(pivotIndex)
    swap(arr, pivotIndex, high)  // Move pivot to end
    var i = low - 1
    
    for (j <- low to high - 1) {  // Don't include pivot
      if (arr(j) <= pivot) {
        i += 1
        swap(arr, i, j)
      }
    }
    swap(arr, i + 1, high)  // Move pivot to final position
    i + 1
  }
  
  // Hoare partition scheme (faster, but more complex)
  def partitionHoare(arr: Array[Int], low: Int, high: Int): Int = {
    val pivot = arr(low)
    var i = low - 1
    var j = high + 1
    
    while (true) {
      do { i += 1 } while (arr(i) < pivot)
      do { j -= 1 } while (arr(j) > pivot)
      
      if (i >= j) return j
      swap(arr, i, j)
    }
    
    j  // Should not reach here
  }
  
  // Different pivot selection strategies
  sealed trait PivotStrategy
  case object FirstElement extends PivotStrategy
  case object LastElement extends PivotStrategy
  case object RandomElement extends PivotStrategy
  case object MedianOfThree extends PivotStrategy
  
  def choosePivot(arr: Array[Int], low: Int, high: Int, strategy: PivotStrategy): Int = strategy match {
    case FirstElement => low
    case LastElement => high
    case RandomElement => low + scala.util.Random.nextInt(high - low + 1)
    case MedianOfThree =>
      val mid = low + (high - low) / 2
      val a = arr(low)
      val b = arr(mid)
      val c = arr(high)
      if ((a <= b && b <= c) || (c <= b && b <= a)) mid
      else if ((b <= a && a <= c) || (c <= a && a <= b)) low
      else high
  }
  
  def sort(arr: Array[Int], strategy: PivotStrategy = MedianOfThree): Array[Int] = {
    val result = arr.clone()
    sortInPlace(result, 0, result.length - 1, strategy)
    result
  }
  
  private def sortInPlace(arr: Array[Int], low: Int, high: Int, strategy: PivotStrategy): Unit = {
    if (low < high) {
      val pivotIndex = choosePivot(arr, low, high, strategy)
      val partitionIndex = partitionLomuto(arr, low, high, pivotIndex)
      
      sortInPlace(arr, low, partitionIndex - 1, strategy)
      sortInPlace(arr, partitionIndex + 1, high, strategy)
    }
  }
}

// Test Quick Sort with different strategies
println("=== Quick Sort Tests ===")
val testArray = testArrays("small").clone()
printArray(testArray, "Original")

println("\nTesting different pivot strategies:")
List(QuickSort.FirstElement, QuickSort.LastElement, QuickSort.RandomElement, QuickSort.MedianOfThree).foreach { strategy =>
  val sorted = QuickSort.sort(testArray.clone(), strategy)
  val strategyName = strategy.toString.split('\'').last
  println(s"$strategyName: ${sorted.mkString("[", ", ", "]")}")
}

// Performance comparison
val largeArray = randomArray(10, 100)
printArray(largeArray.take(10), "Large array (first 10)")
println(s"Quick Sort result (${largeArray.length} elements): ${isSorted(QuickSort.sort(largeArray))}")
println()

## 🔗 Merge Sort (O(n log n) - Stable)

Efficient, stable, and predictable sorting algorithm with consistent performance.

In [ ]:
// Merge Sort: Top-down recursive implementation
object MergeSort {
  
  def sort(arr: Array[Int]): Array[Int] = {
    val result = arr.clone()
    mergeSort(result, 0, result.length - 1)
    result
  }
  
  private def mergeSort(arr: Array[Int], left: Int, right: Int): Unit = {
    if (left < right) {
      val mid = left + (right - left) / 2
      
      mergeSort(arr, left, mid)
      mergeSort(arr, mid + 1, right)
      
      merge(arr, left, mid, right)
    }
  }
  
  private def merge(arr: Array[Int], left: Int, mid: Int, right: Int): Unit = {
    val n1 = mid - left + 1
    val n2 = right - mid
    
    val leftArr = Array.ofDim[Int](n1)
    val rightArr = Array.ofDim[Int](n2)
    
    // Copy data to temp arrays
    for (i <- 0 until n1) leftArr(i) = arr(left + i)
    for (j <- 0 until n2) rightArr(j) = arr(mid + 1 + j)
    
    // Merge the temp arrays
    var i = 0
    var j = 0
    var k = left
    
    while (i < n1 && j < n2) {
      if (leftArr(i) <= rightArr(j)) {
        arr(k) = leftArr(i)
        i += 1
      } else {
        arr(k) = rightArr(j)
        j += 1
      }
      k += 1
    }
    
    // Copy remaining elements
    while (i < n1) {
      arr(k) = leftArr(i)
      i += 1
      k += 1
    }
    
    while (j < n2) {
      arr(k) = rightArr(j)
      j += 1
      k += 1
    }
  }
  
  // In-place merge sort variant (simplified)
  def sortInPlace(arr: Array[Int]): Unit = {
    mergeSort(arr, 0, arr.length - 1)
  }
}

// Test Merge Sort
println("=== Merge Sort Tests ===")
val testArray = testArrays("small").clone()
printArray(testArray, "Original")

val sortedMerge = MergeSort.sort(testArray)
printArray(sortedMerge, "Merge Sorted")
println(s"Is sorted: ${isSorted(sortedMerge)}")
println(s"Stable sort: ${testArrays("small").mkString(",")} → ${sortedMerge.mkString(",")}")

// Performance test
val largeArray = randomArray(10000)
val startTime = System.nanoTime()
val sortedLarge = MergeSort.sort(largeArray)
val endTime = System.nanoTime()
println(f"Merge Sort 10K elements: ${((endTime - startTime) / 1e6).toInt} ms")
println(s"Result is sorted: ${isSorted(sortedLarge)}")
println()

## 🔔 Insertion Sort (O(n²) worst, O(n) best - Stable)

Excellent for small arrays and nearly sorted data.

In [ ]:
// Insertion Sort: Builds sorted array one element at a time
def insertionSort(arr: Array[Int]): Array[Int] = {
  val result = arr.clone()
  
  for (i <- 1 until result.length) {
    val key = result(i)
    var j = i - 1
    
    // Move elements that are greater than key
    while (j >= 0 && result(j) > key) {
      result(j + 1) = result(j)
      j -= 1
    }
    
    result(j + 1) = key
  }
  
  result
}

// Binary Insertion Sort: Uses binary search to find insertion point
def binaryInsertionSort(arr: Array[Int]): Array[Int] = {
  val result = arr.clone()
  
  for (i <- 1 until result.length) {
    val key = result(i)
    
    // Binary search for insertion point
    val insertionPoint = binarySearchInsertionPoint(result, key, 0, i - 1)
    
    // Shift elements and insert
    System.arraycopy(result, insertionPoint, result, insertionPoint + 1, i - insertionPoint)
    result(insertionPoint) = key
  }
  
  result
}

private def binarySearchInsertionPoint(arr: Array[Int], key: Int, left: Int, right: Int): Int = {
  var low = left
  var high = right
  
  while (low <= high) {
    val mid = low + (high - low) / 2
    if (arr(mid) > key) {
      if (mid == 0 || arr(mid - 1) <= key) {
        return mid
      }
      high = mid - 1
    } else {
      if (mid == right || arr(mid + 1) > key) {
        return mid + 1
      }
      low = mid + 1
    }
  }
  
  low
}

println("=== Insertion Sort Tests ===")

// Performance on nearly sorted data
val nearlySorted = (1 to 100).toArray ++ Array(0)
val startTime = System.nanoTime()
val sortedInsert = insertionSort(nearlySorted)
val endTime = System.nanoTime()
println(f"Insertion Sort (nearly sorted): ${((endTime - startTime) / 1e3).toInt} μs")
println(s"Result: [${sortedInsert.take(5).mkString(",")}...${sortedInsert.takeRight(5).mkString(",")}]")

// Comparison with regular sort
val randomData = randomArray(50)
val normalInsert = insertionSort(randomData.clone())
val binaryInsert = binaryInsertionSort(randomData.clone())
println(s"Regular vs Binary: both sorted=${isSorted(normalInsert) && isSorted(binaryInsert)}")

// Efficiency on small arrays
val smallArrays = (1 to 5).map(_ => randomArray(10))
smallArrays.foreach(arr => assert(isSorted(insertionSort(arr))))
println("Small arrays: All correctly sorted")
println()

## 🎯 Interview Questions: Sorting Algorithm Choice

### **When to Use Each Sorting Algorithm:**

| Scenario | Best Algorithm | Why |
|----------|----------------|-----|
| **Nearly sorted data** | Insertion Sort | O(n) vs O(n²) for others |
| **Small arrays (n<10)** | Insertion Sort | Low overhead |
| **Guaranteed O(n log n)** | Merge Sort | Predictable performance |
| **Memory constrained** | Heap Sort | O(1) extra space |
| **Stable sort needed** | Merge Sort/Bubble/Insertion | Preserves relative order |
| **Large, random data** | Quick Sort | Fastest in practice |
| **Integer range known** | Counting Sort | O(n) time |
| **Need partial ordering** | Quick Select | Fast k-th element |

### **Common Interview Questions:**

**Q: Explain why QuickSort is preferred over MergeSort in practice?**
- **Cache efficiency**: Quicksort accesses memory sequentially, better cache performance
- **In-place sorting**: MergeSort requires O(n) extra space
- **Constant factors**: Quicksort has better constant factors
- **Practical performance**: On real hardware, QuickSort often outperforms MergeSort

**Q: How would you sort strings by length, then lexicographically?**
- **Custom comparator**: `(a:String, b:String) => (a.length compare b.length) orElse (a compare b)`
- **Functional approach**: `strings.sortBy(s => (s.length, s))`
- **Stability matters**: Need stable sort for meaningful secondary ordering

**Q: Implement a stable sort in-place**
- **Challenge**: Most efficient algorithms (QuickSort) are not stable
- **Solution**: Use Merge Sort variant or Bubble Sort for small arrays
- **Trade-off**: Performance vs stability requirement
